In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm

In [3]:
pdbs_to_be_tested = ["1VRT", "1HQE", "1HHP", "1AJX", "1J6Z", "1ATN", "1ANF",
                     "1OMP", "1GGG", "1WDN", "2LAO", "1G0X", "1P7Q", "1EK9",
                     "1CRZ"]

In [11]:
molsys = msm.convert('1ATN')

[2]


/home/diego/projects@dprada/MolSysMT/molsysmt/item/mmtf_MMTFDecoder/to_molsysmt_Topology.py:31: UserWarning: The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: https://github.com/uibcdf/MolSysMT/issues/33
  warnings.warn(warning_message)


In [18]:
msm.info(molsys, element='entity')

index,name,type,n atoms,n groups,n components,n chains,n molecules
0,Actin,protein,2911,373,1,1,1
1,Deoxyribonuclease i,protein,2034,258,1,1,1
2,Beta-d-mannopyranose-(1-4)-2-acetamido-2-deoxy-beta-d-glucopyranose-(1-4)-2-acetamido-2-deoxy-beta-d-glucopyranose,oligosaccharide,39,3,1,1,1
3,Calcium ion,ion,4,4,4,4,4
4,Adenosine-5'-triphosphate,small molecule,31,1,1,1,1


In [10]:
msm.info(molsys, element='molecule')

index,name,type,n atoms,n groups,n components,chain index,entity index,entity name
0,Actin,protein,2911,373,1,0,0,Actin
2,Beta-d-mannopyranose-(1-4)-2-acetamido-2-deoxy-beta-d-glucopyranose-(1-4)-2-acetamido-2-deoxy-beta-d-glucopyranose,oligosaccharide,0,0,0,[],1,Deoxyribonuclease i
3,Calcium ion,ion,2073,261,1,[1 2],3,Calcium ion
7,Adenosine-5'-triphosphate,small molecule,1,1,1,3,4,Adenosine-5'-triphosphate
4,Calcium ion,ion,1,1,1,5,3,Calcium ion
5,Calcium ion,ion,1,1,1,6,3,Calcium ion
6,Calcium ion,ion,1,1,1,7,3,Calcium ion


In [6]:
mmtf = msm.convert('1ATN', to_form='mmtf.MMTFDecoder')

In [8]:
msm.info(molsys, element='molecule')

index,name,type,n atoms,n groups,n components,chain index,entity index,entity name
0,Actin,protein,2911,373,1,0,0,Actin
2,Beta-d-mannopyranose-(1-4)-2-acetamido-2-deoxy-beta-d-glucopyranose-(1-4)-2-acetamido-2-deoxy-beta-d-glucopyranose,oligosaccharide,0,0,0,[],1,Deoxyribonuclease i
3,Calcium ion,ion,2073,261,1,[1 2],3,Calcium ion
7,Adenosine-5'-triphosphate,small molecule,1,1,1,3,4,Adenosine-5'-triphosphate
4,Calcium ion,ion,1,1,1,5,3,Calcium ion
5,Calcium ion,ion,1,1,1,6,3,Calcium ion
6,Calcium ion,ion,1,1,1,7,3,Calcium ion


In [9]:
msm.info(mmtf, element='molecule')

/home/diego/projects@dprada/MolSysMT/molsysmt/item/mmtf_MMTFDecoder/to_molsysmt_Topology.py:31: UserWarning: The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: https://github.com/uibcdf/MolSysMT/issues/33
  warnings.warn(warning_message)
/home/diego/projects@dprada/MolSysMT/molsysmt/item/mmtf_MMTFDecoder/to_molsysmt_Topology.py:31: UserWarning: The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: https://github.com/uibcdf/MolSysMT/issues/33
  warnings.warn(warning_message)
/home/diego/projects@dprada/MolSysMT/molsysmt/item/mmtf_MMTFDecoder/to_molsysmt_Topology.py:31: UserWarning: The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: h

IndexError: index 0 is out of bounds for axis 0 with size 0